# Tensorboard Setup

https://www.tensorflow.org/tensorboard/get_started

Run all code cells below, then run on console:

`tensorboard --logdir logs/train`

In [ ]:
import sys
import os
sys.path.append("..")

In [ ]:
# Setup
import reproducibility

# both can change
# Environment
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
# from keras.utils.vis_utils import plot_model
from keras.utils import plot_model

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import json
import matplotlib.pyplot as plt
import tensorflow as tf
import keras.backend as K
# from keras.utils.vis_utils import plot_model
from scipy.interpolate import LinearNDInterpolator, interpn
from scipy.optimize import root
import pickle, os
from utils import hash2
%matplotlib inline

# Local modules for handling data and running moisture models
import data_funcs as datf
from data_funcs import format_raws, retrieve_raws, format_precip, fixnan, load_and_fix_data
from data_funcs import raws_data, synthetic_data, plot_data, check_data, rmse_data, to_json, from_json
import moisture_models as mod
from moisture_rnn import run_case, run_rnn, create_RNN_2, staircase, create_rnn_data, train_rnn, rnn_predict


In [ ]:
# Data
# Change directory for data read/write

dict_file='../data/raws_CO_202306.pickle' # input path of FMDA dictionaries
reproducibility_file='../data/reproducibility_dict.pickle'

# read test datasets
test_dict={}
test_dict.update(load_and_fix_data(dict_file))
print(test_dict.keys())

repro_dict={}
repro_dict.update(load_and_fix_data(reproducibility_file))
print(repro_dict.keys())

In [ ]:
# Build Case Data
id = "CPTC2_202306010000"
case_data=test_dict[id]
case_data["hours"]=len(case_data['fm'])
case_data["h2"]=int(24*20)

In [ ]:
# Linear Activation Params
params={'id':1,
        'purpose':'test 1',
        'cases':'all',
        'scale':1,        # every feature in [0, scale]
        'rain_do':True,
        'verbose':False,
        'timesteps':5,
        'activation':['linear','linear'],
        'hidden_units':1,  
        'dense_units':1,    # do not change
        'dense_layers':1,   # do not change
        'centering':[0.0,0.0],  # should be activation at 0
        'DeltaE':[0,-1],    # bias correction
        'synthetic':False,  # run also synthetic cases
        'T1': 0.1,          # 1/fuel class (10)
        'fm_raise_vs_rain': 2.0,         # fm increase per mm rain 
        'train_frac':0.5,  # time fraction to spend on training
        'epochs':1000,
        'verbose_fit':False,
        'verbose_weights':False,
}

In [ ]:
# Callback
log_dir = "logs"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
reproducibility.set_seed() # Set seed for reproducibility
rnn_dat = create_rnn_data(case_data,params)
model1 = train_rnn(
    rnn_dat,
    params,
    rnn_dat['hours'],
    fit=True,
    callbacks = [tensorboard_callback]
)
case_data['m'] = rnn_predict(model1, params, rnn_dat)
fit_linear = case_data['m']

In [ ]:
plot_data(case_data,title2='Trained RNN Linear')